# SYSM 6302 - Lab 2
Jonas Wagner, jrw200000



### Section 6.9: Degree

An example of networks where a node having a high degree is a good thing may be a social network in which edges represent freindships. In such a network, having a high degree indicates you have many friends.

An example of networks where a node having a high degree is a bad thing may be a highway system in which nodes are interchanges of highways. In such a network, having a high degree indicates a lot of highways being connected through that interchange (in other words, a lot of traffic).

An example of a low degree node that may be important would be a single node that solely connects two highly connected (technically not) components. This node may have a low degree, but is vital for allowing paths between two subsets of a graph.

### Snub Code

In [17]:
import networkx as nx
import sys
sys.path.append('../d3networkx/')
import d3networkx as d3nx
from d3graph import D3Graph, D3DiGraph
from numpy import *
from time import time
import asyncio

def square_grid(n,d3,G,x0=100,y0=100,w=50):
    if G is None:
        G = D3Graph()
    # find the dimensions for the grid that are as close as possible
    num_rows = int(floor(sqrt(n)))
    while n % num_rows != 0:
        num_rows += 1
    num_cols = int(n/num_rows)
    
    # Add all the nodes
    G.add_nodes_from(range(n))
        
    # Add the edges and position the nodes
    for i in range(num_rows):
        for j in range(num_cols):
            n = num_cols*i + j
            d3.position_node(n,x0+i*w,y0+j*w)
            if i < num_rows-1:
                G.add_edge(n,n+num_cols) # add edge down
            if j < num_cols-1:
                G.add_edge(n,n+1) # add edge right

async def propagate(G,d3,x,steps,slp=0.5,keep_highlights=False,update_at_end=False):
    interactive = d3.interactive
    d3.set_interactive(False)
    A = nx.adjacency_matrix(G).todense().T  # adjacency matrix
    d3.highlight_nodes_by_index(list(where(x>0)[0]))
    d3.update()
    await asyncio.sleep(slp)
    cum_highlighted = sign(x)
    for i in range(steps): # the brains
        x = sign(dot(A,x)) # the brains
        cum_highlighted = sign(cum_highlighted+x)
        if not update_at_end:
            if not keep_highlights:
                d3.clear_highlights()
            d3.highlight_nodes_by_index(list(where(x>0)[0]))
            d3.update()
            await asyncio.sleep(slp)
    if update_at_end:
        if not keep_highlights:
            d3.clear_highlights()
            d3.highlight_nodes_by_index(list(where(x>0)[0]))
        else:
            d3.highlight_nodes_by_index(list(where(cum_highlighted>0)[0]))
        d3.update()
    d3.set_interactive(interactive)
    if keep_highlights:
        return cum_highlighted
    else:
        return x 

This next line starts up the visualizer. It will start some background code that sends data to the visualizer and then it will open a new browser window where the visualizer will live. Once you have the visualizer running, you can leave it running for the entire session, so don't re-run this block. If you close the `visualizer.html` (or hit refresh), you will need to reestablish this connection. In this case, you should click the refresh button in the Jupyter notebook (not for the webpage) to restart the kernel (which will clear your variables and Python environment).

In [18]:
d3 = await d3nx.create_d3nx_visualizer()
#d3 = await d3nx.create_d3nx_visualizer(canvas_size=(1200,1000))

websocket server started...networkx connected...visualizer connected...

## Grid Network

In [3]:
d3.clear()
d3.set_interactive(False)
G = D3Graph()
d3.set_graph(G)
square_grid(144,d3,G,x0=75,y0=70)
d3.update()

In [4]:
x = zeros((G.number_of_nodes(),1))
x[0] = 1
await propagate(G,d3,x,10,slp=1);

## Directed Network

In [5]:
d3.clear()
G = D3DiGraph(nx.read_weighted_edgelist('lab2.edgelist',create_using=nx.DiGraph))
d3.set_graph(G)
d3.update()

In [6]:
G.add_edge('test','3')
# d3.set_graph(G)
# d3.update()

In [7]:
d3.set_interactive(True)

In [8]:
G.remove_node('4')

In [9]:
# code to propagate

In [10]:
# code to find the out-component of node 1

## E. coli Protein Network

In [11]:
d3.clear()
G = D3DiGraph(nx.read_weighted_edgelist('ecoli.edgelist',create_using=nx.DiGraph))
d3.set_interactive(False)
d3.set_graph(G)
d3.set_interactive(True)
d3.update()
print('Ecoli has %i nodes.' % G.number_of_nodes())

Ecoli has 418 nodes.


In [12]:
# code to find the out-component of node 2

In [13]:
# code to find the out-component of node 16

In [14]:
#print(nx.diameter(G)) # does not work!

def diameter2(G):
    spaths = dict(nx.all_pairs_shortest_path(G))
    # fill in!
    return diameter, path

# use the new diameter function here

## Section 6.12: Flows & Cut Sets

In [15]:
def worst_graph(n):
    # to do
    return G

def best_graph(n):
    # to do
    return G    

# create graph
start_time = time()
# perform min cut
print('min cut took %1.2f seconds' % (time() - start_time))

min cut took 0.00 seconds


In [16]:
G = nx.read_gml('pert.gml','name')

# find longest path